# Postoperative AKI Prediction

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
import scipy.stats as stats
from collections import Counter
from tqdm import tqdm
import os
import shutil

matplotlib.rcParams['font.sans-serif'] = ['Arial Unicode MS']


pd.set_option('max_colwidth',200)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

shap.initjs()


In [2]:
df=pd.read_excel("final_data.xlsx")
# df.head()

# Data Preprocessing

In [ ]:
print(df.shape)
df.info()

In [4]:
# hight and weight should be float or int instead of object
df['Weight'] = pd.to_numeric(df['Weight'], errors='coerce')
df['Height'] = pd.to_numeric(df['Height'], errors='coerce')

In [5]:
# df.describe().T

# we can see many outliers in height and weight, we will deal with them by visualizing the distribution

In [ ]:
# count the postive samples
df['aki'].value_counts()

In [ ]:
df[df['aki'] == 1]['Surgery type'].value_counts()

In [ ]:
to_drop = [
       # delete time related columns
       'Surgery begin time', 'Surgery end time', 'Test time',
       # delete description columns
       'Surgery name']

df.drop(to_drop, axis=1, inplace=True)

df.shape

In [ ]:
df.columns

## Delete Samples with Missing Values

In [ ]:
# missing values for negative samples

def missing_rows(df):
    '''
    calculate the missing values of each row
    input:
        df: the dataframe
    output:
        missing_percent: the missing percentage of each row
        missing_count: the number of missing values of each row
    '''
    missing_row = df.isna()
    missing_row = missing_row[missing_row.any(axis=1)]
    missing_percent = missing_row.mean(axis=1)
    missing_count = missing_percent.value_counts()
    return missing_percent, missing_count

missing_percent0, missing_count0 = missing_rows(df[df['aki'] == 0])
print('Missing percentage of negative samples:')
print(missing_count0)
print('total number of negative samples with missing values:', sum(missing_count0))

In [ ]:
# drop the negative rows with missing values
df = df.drop(missing_percent0.index, axis=0)

df.shape

In [ ]:
# missing values for positive samples

missing_percent1, missing_count1 = missing_rows(df[df['aki'] == 1])
print('Missing percentage of positive samples:')
print(missing_count1)
print('total number of positive samples with missing values:', sum(missing_count1))

# drop the rows with missing percentage > 0.2
to_drop = missing_percent1[missing_percent1 > 0.2].index
print(f'drop the rows with missing percentage > 0.2: {len(to_drop)} rows')
df.drop(to_drop, inplace=True)
df.shape

In [ ]:
def missing_columns(df):
    """
    Return the missing values of each column
    input:
        df: the dataframe
    output:
        missing_values: the missing count and percentage of each column
    """
    missing_number = df.isnull().sum().sort_values(ascending=False)
    missing_percent = (df.isnull().sum()/df[df['aki'] == 1].shape[0]).sort_values(ascending=False)
    missing_values = pd.concat([missing_number, missing_percent], axis=1, keys=['Missing_Number', 'Missing_Percent'])
    missing_values = missing_values[missing_values['Missing_Number'] > 0]
    if missing_values.shape[0] == 0:
        print('No missing values')
    else:
        return missing_values
    
missing_col = missing_columns(df)
missing_col

In [ ]:
# drop the column with missing percent > 0.2
dropped_col = missing_col[missing_col['Missing_Percent'] > 0.2].index
print(dropped_col)
df.drop(dropped_col, axis=1, inplace=True)
df.shape

## Delete Outliers


In [ ]:
df.select_dtypes('object').columns #discrete
df.select_dtypes('int64').columns #continuous
df.select_dtypes('float64').columns #continuous

In [ ]:
numeric_cols = df.select_dtypes('float64').columns

# Define the number of rows and columns for the subplots
n_cols = 3
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols  # Calculate the number of rows needed

# Create a figure and a grid of subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 4))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plot each numeric column in a subplot
for i, col in enumerate(numeric_cols):
    sns.histplot(df[col], bins=30, kde=True, ax=axes[i])
    axes[i].set_title(f'Distribution of {col}')

# Remove any empty subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

# Adjust layout
plt.tight_layout()
plt.show()



In [17]:
# for each numerical column, find the highest and lowest 10 values and the corresponding 'aki'

outliers = pd.DataFrame()
for col in numeric_cols:
    highest_10 = df[[col, 'aki']].nlargest(10, col)
    lowest_10 = df[[col, 'aki']].nsmallest(10, col)
    
    highest_10['feature'] = col
    highest_10['type'] = 'highest'
    highest_10 = highest_10.rename(columns={col: 'value'})
    
    lowest_10['feature'] = col
    lowest_10['type'] = 'lowest'
    lowest_10 = lowest_10.rename(columns={col: 'value'})
    
    outliers = pd.concat([outliers, highest_10, lowest_10])

outliers = outliers[['feature', 'type', 'value', 'aki']]
# outliers.to_excel('outliers.xlsx', index=False)


In [ ]:
# deal with outliers

# drop height higher than 200 or lower than 100 and print number of dropped rows
to_drop_height = df[(df['Height'] > 200) | (df['Height'] < 100)]
print(f'Number of dropped rows due to height outliers: {to_drop_height.shape[0]}')
df.drop(to_drop_height.index, inplace=True)

# drop the rows with weight less than 40
to_drop_weight = df[df['Weight'] < 40]
print(f'Number of dropped rows due to weight outliers: {to_drop_weight.shape[0]}')
df.drop(to_drop_weight.index, inplace=True)

# delete the rows whose surgery time is less than 10 minues
to_drop_time = df[df['Operation time (mins)'] < 10]
print(f'Number of dropped rows due to operation time outliers: {to_drop_time.shape[0]}')
df.drop(to_drop_time.index, inplace=True)

to_drop_bmi = df[(df['BMI'] < 10) | (df['BMI'] > 50)]
print(f'Number of dropped rows due to BMI outliers: {to_drop_bmi.shape[0]}')
df.drop(to_drop_bmi.index, inplace=True)

## Fill in NA values

In [19]:
average_weight = df['Weight'].mean()
df['Weight'] = df['Weight'].fillna(average_weight)

average_height = df['Height'].mean()
df['Height'] = df['Height'].fillna(average_height)

In [20]:
def calculate_bmi(row):
    height = row['Height']
    weight = row['Weight']
    if pd.isna(height) or pd.isna(weight):
        return np.nan
    else:
        return weight / ((height / 100) ** 2)

df['BMI'] = df.apply(calculate_bmi, axis=1)  
df.drop(['Height', 'Weight'], inplace=True, axis=1)

In [21]:
# use knn to fill in NA value for positive samples
from sklearn.impute import KNNImputer

numeric_for_impute = ['Age (years)', 'BMI', 'Serum creatinine (μmol/L)', 'Serum albumin (g/L)', 'Serum carbon dioxide (mmol/L)', 'Urea (mmol/L)',
       'Serum potassium (mmol/L)', 'Serum sodium (mmol/L)', 'Neutrophils (*10^9/L)', 'Lymphocyte (*10^9/L)',
       'Monocyte (*10^9/L)', 'Platelet (*10^9/L)', 'NLR', 'PLR', 'PNI', 'MLR', 'SII', 'UCR']

df_b4_impute = df.copy()

# Encode the categorical column "Gender"
encoder = preprocessing.OrdinalEncoder()
df1 = df[df['aki'] == 1].copy()
df1['Gender_encoded'] = encoder.fit_transform(df1[['Gender']])

# Combine the numeric columns with the encoded "Gender" column
columns_for_imputation = numeric_for_impute + ['Gender_encoded']
df1_for_imputation = df1[columns_for_imputation]

# Apply KNNImputer
imputer = KNNImputer(n_neighbors=5)
df1_imputed = pd.DataFrame(imputer.fit_transform(df1_for_imputation), columns=columns_for_imputation)

# Round the imputed "Gender" column and convert to int
df1_imputed['Gender_encoded'] = np.round(df1_imputed['Gender_encoded']).astype(int)

# Decode the imputed "Gender" column
df1['Gender_imputed'] = encoder.inverse_transform(df1_imputed[['Gender_encoded']]).flatten()

# Update the original dataframe with the imputed values
df1_imputed.index = df1.index
df1.update(df1_imputed[numeric_for_impute])
df1['Gender'] = df1['Gender_imputed']

# Update the original dataframe
df.update(df1)

# Drop the temporary columns
df1.drop(columns=['Gender_encoded', 'Gender_imputed'], inplace=True)

# Create a comparison DataFrame to show the differences
comparison_df = df_b4_impute.copy()
comparison_df['Gender_imputed'] = df['Gender']
for col in numeric_for_impute:
    comparison_df[col + '_imputed'] = df[col]

# Create the desired column order with original columns next to imputed columns
ordered_columns = []

ordered_columns.append('Gender')
ordered_columns.append('Gender_imputed')

for col in numeric_for_impute:
    ordered_columns.append(col)
    ordered_columns.append(col + '_imputed')

# Reorder the columns in the comparison DataFrame
comparison_df = comparison_df[ordered_columns]

comparison_df_highlight = comparison_df[(df_b4_impute != df).any(axis=1)]

# comparison_df_highlight


In [ ]:
missing_rows(df)

## Feature Transformation

In [ ]:
df.select_dtypes('object').columns #discrete
df.select_dtypes('int64').columns #continuous
df.select_dtypes('float64').columns #continuous

In [ ]:
# select the columns with surgery type and operation time
category_cols =['Surgery type', 'Gender', 'Hypertension', 'Diabetes mellitus', 'Coronary artery disease', 'COPD', 'Cancer']

numeric_cols =['Operation time (mins)', 'Age (years)', 'BMI', 'Serum creatinine (μmol/L)',
       'Serum albumin (g/L)', 'Serum carbon dioxide (mmol/L)', 'Urea (mmol/L)',
       'Serum potassium (mmol/L)', 'Serum sodium (mmol/L)',
       'Neutrophils (*10^9/L)', 'Lymphocyte (*10^9/L)', 'Monocyte (*10^9/L)',
       'Platelet (*10^9/L)', 'NLR', 'PLR', 'PNI', 'MLR', 'SII',
       'UCR']

target = 'aki'

df = df.reset_index(drop=True)

original_df = df.copy()

df = df[category_cols + numeric_cols + [target]]

df.shape


In [25]:
# df[category_cols].head()

In [26]:
df['Surgery type'] = df['Surgery type'].map({'TEVAR': 1, 'EVAR': 0, 'OSR': 2})
df['Gender'] = df['Gender'].map({'M': 1, 'F': 0})
df['Hypertension'] = df['Hypertension'].map({'Y': 1, 'N': 0})
df['Diabetes mellitus'] = df['Diabetes mellitus'].map({'Y': 1, 'N': 0})
df['Coronary artery disease'] = df['Coronary artery disease'].map({'Y': 1, 'N': 0})
df['COPD'] = df['COPD'].map({'Y': 1, 'N': 0})
df['Cancer'] = df['Cancer'].map({'Y': 1, 'N': 0})

# df[category_cols].head()

In [ ]:
# Plotting the distplots without any transformation
# To know the scale of how skewed the data is we can plot a probability distribution plot that exactly describes what transformations are needed to be performed

def plot_feature_distribution(df, numeric_cols):
    '''
    plot the distribution of each numeric column
    input:
        df: the dataframe
        numeric_cols: the numeric columns
    output:
        print (1) the distribution of each numeric column and the proportion of positive samples in each bin
        print (2) the probability plot of each numeric column
    '''
    # Define the number of rows and columns for the subplots
    n_cols = 1
    n_rows = (len(numeric_cols) + n_cols - 1) // n_cols  # Calculate the number of rows needed
    bin_count = 15
    font_size = 20

    # Create a figure and a grid of subplots
    fig, axes = plt.subplots(n_rows, n_cols * 2, figsize=(20, n_rows * 8))
    axes = axes.flatten()

    # Plot each numeric column in a subplot
    for i, col in enumerate(numeric_cols):
        # First subplot: Histogram
        sns.histplot(df[col], bins=bin_count, kde=True, ax=axes[2 * i])
        axes[2 * i].set_title(f'Distribution of {col}', fontsize=font_size)
        axes[2 * i].set_xlabel(col, fontsize=font_size)
        axes[2 * i].set_ylabel('Frequency', fontsize=font_size)
        axes[2 * i].tick_params(labelsize=font_size)
        
        # Calculate the proportion of positive samples in each bin
        bins = np.linspace(df[col].min(), df[col].max(), bin_count+1)
        df['bin'] = pd.cut(df[col], bins=bins, include_lowest=True)
        bin_mean = df.groupby('bin')['aki'].mean()
        bin_centers = 0.5 * (bins[:-1] + bins[1:])

        # Overlay the proportion of positive samples
        ax2 = axes[2 * i].twinx()
        ax2.plot(bin_centers, bin_mean, 'r-', marker='o')
        ax2.set_ylabel('Proportion of Positive Samples', color='r', fontsize=font_size)
        ax2.tick_params(axis='y', labelcolor='r', labelsize=font_size)

        # Extract the color used in the histogram plot
        color = axes[2 * i].patches[0].get_facecolor()
        
        # Second subplot: Probability plot
        (osm, osr), (slope, intercept, r) = stats.probplot(df[col], dist="norm")
        axes[2 * i + 1].plot(osm, osr, 'o', color=color)  # Use the extracted color
        axes[2 * i + 1].plot(osm, slope * osm + intercept, 'b-')
        axes[2 * i + 1].set_title(f'Probability Plot of {col}', fontsize=font_size)
        axes[2 * i + 1].tick_params(labelsize=font_size)



    # Remove any empty subplots
    for j in range(2 * i + 2, len(axes)):
        fig.delaxes(axes[j])

    # Adjust layout
    plt.tight_layout()
    plt.show()
    df.drop(columns=['bin'], inplace=True)

# Example usage
plot_feature_distribution(df, numeric_cols)

In [28]:
# use log1p transformation for some of the skewed features

skewed = ['Operation time (mins)', 'Serum creatinine (μmol/L)', 'Urea (mmol/L)', 'Neutrophils (*10^9/L)', 'Lymphocyte (*10^9/L)', 'Monocyte (*10^9/L)', 'Platelet (*10^9/L)', 'NLR', 'PLR', 'MLR', 'SII']


transformer = ColumnTransformer(
    transformers=[
        ('log1p', preprocessing.FunctionTransformer(np.log1p), skewed)
    ],
    remainder='passthrough'
)

# Apply the transformer
transformed_data = transformer.fit_transform(df)

# Convert the transformed data back to a DataFrame
transformed_df = pd.DataFrame(transformed_data, columns=skewed + [col for col in df.columns if col not in skewed])

# Ensure the column order matches the original DataFrame
df = transformed_df[df.columns]

# df.head()


In [ ]:
plot_feature_distribution(df, numeric_cols)


# Machine Learning Models

In [30]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier


import lightgbm as lgb
import xgboost as xgb
import random

In [ ]:
intraoperative_features = ['Operation time (mins)', 'Surgery type']
intraoperative = df[intraoperative_features].copy()
X = df.drop(columns=[target]+intraoperative_features).copy()
y = df[target].copy()
new_X = pd.concat([X, intraoperative], axis=1)
numeric_noTime = [col for col in numeric_cols if col != 'Operation time (mins)']


print(X.shape)
print(y.value_counts())
# X.head()


## Supporting Functions

In [32]:
# test train test split
from scipy.stats import mannwhitneyu, chi2_contingency

def test_splitting(original_df, train_index, test_index):

    '''
    use mannwhitneyu test for continuous features and chi square test for categorical features to test whether the distribution of the training set and testing set are different
    if p value < 0.05, then the distribution of the training set and testing set are significantly different
    assume the data is not normally distributed
    input:
        original_df: the original dataframe
        train_index: the index of the training set
        test_index: the index of the testing set
    output:
        True/False, spreadsheet_df
    '''

    np.random.seed(42)


    train_set = original_df.iloc[train_index, :]
    test_set = original_df.iloc[test_index, :]

    spreadsheet = []

    # Patient population row
    spreadsheet.append({
        'Feature': 'Patient Population',
        'All': len(original_df),
        'Training Set': len(train_set),
        'Testing Set': len(test_set),
        'P-value': ''
    })

    # Process categorical columns
    for col in category_cols:
        if col != 'Surgery type':
            counts_all = original_df[col].value_counts()
            counts_train = train_set[col].value_counts()
            counts_test = test_set[col].value_counts()
            
            if col == 'Gender':
                cat = 'M'
            else:
                cat = 'Y'
            count_all = counts_all.get(cat, 0)
            count_train = counts_train.get(cat, 0)
            count_test = counts_test.get(cat, 0)
            if count_all == 0:
                p_value = 1
            else:
                p_value = chi2_contingency(
                    [[count_train, len(train_set) - count_train],
                        [count_test, len(test_set) - count_test]]
                )[1]
            spreadsheet.append({
                'Feature': f"{col} ({cat})",
                'All': f"{count_all} ({count_all/len(df)*100:.2f})",
                'Training Set': f"{count_train} ({count_train/len(train_set)*100:.2f})",
                'Testing Set': f"{count_test} ({count_test/len(test_set)*100:.2f})",
                'P-value': f"{p_value:.4f}"
            })
        
        # surgery type have 3 categories, we need to deal with it seperately
        else:
            counts_all = original_df[col].value_counts()
            counts_train = train_set[col].value_counts()
            counts_test = test_set[col].value_counts()
            
            for cat in counts_all.index:
                count_all = counts_all[cat]
                count_train = counts_train.get(cat, 0)
                count_test = counts_test.get(cat, 0)
                p_value = chi2_contingency(
                    [[count_train, len(train_set) - count_train],
                    [count_test, len(test_set) - count_test]]
                )[1]
                spreadsheet.append({
                    'Feature': f"{col} ({cat})",
                    'All': f"{count_all} ({count_all/len(df)*100:.2f})",
                    'Training Set': f"{count_train} ({count_train/len(train_set)*100:.2f})",
                    'Testing Set': f"{count_test} ({count_test/len(test_set)*100:.2f})",
                    'P-value': f"{p_value:.4f}"
                })

    # Process numerical columns
    for col in numeric_cols:
        all_med = original_df[col].median()
        all_min = original_df[col].min()
        all_max = original_df[col].max()
        train_med = train_set[col].median()
        train_min = train_set[col].min()
        train_max = train_set[col].max()
        test_med = test_set[col].median()
        test_min = test_set[col].min()
        test_max = test_set[col].max()
        
        p_value = mannwhitneyu(train_set[col], test_set[col])[1]
        
        spreadsheet.append({
            'Feature': col,
            'All': f"{all_med:.2f} ({all_min:.2f}-{all_max:.2f})",
            'Training Set': f"{train_med:.2f} ({train_min:.2f}-{train_max:.2f})",
            'Testing Set': f"{test_med:.2f} ({test_min:.2f}-{test_max:.2f})",
            'P-value': f"{p_value:.4f}"
        })

    # Convert to DataFrame for display or export
    spreadsheet_df = pd.DataFrame(spreadsheet)

    # check whether there's any p value less than 0.05 and ignore '' string
    significant_df = spreadsheet_df[spreadsheet_df['P-value'].apply(lambda x: x != '' and float(x) < 0.05)]
    significant = not significant_df.empty
    
    return significant, spreadsheet_df

In [33]:

def calculate_sample(y_test, y_prob):
    '''
    calculate the metrics for the test set
    input:
        y_test: the true labels
        y_prob: the predicted probabilities
    output:
        auc, recall, precision, accuracy, optimal_threshold, y_pred
    '''

    # Determine the optimal threshold using Youden's J Statistic
    fpr, tpr, thresholds = roc_curve(y_test, y_prob)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    y_pred = (y_prob > optimal_threshold).astype(int)

    # Calculate metrics
    auc = roc_auc_score(y_test, y_prob)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=1)
    accuracy = accuracy_score(y_test, y_pred)

    return auc, recall, precision, accuracy, optimal_threshold, y_pred

def save_ci(boostrap, alpha):
    '''
    save the confidence interval in the format of (lower, upper)
    input:
        boostrap: the bootstrapped metrics
        alpha: the significance level
    output:
        the confidence interval in the format of (lower, upper)
    '''
    lower, upper = np.percentile(boostrap, [(alpha / 2) * 100, (1 - alpha / 2) * 100])
    return str('(' + str(round(lower, 3)) + '-' + str(round(upper, 3)) + ')')

def calculate_metrics(y_test, y_prob, ci=False, n_bootstrap=1000, alpha=0.05):

    '''
    calculate the metrics for the test set
    input:
        y_test: the true labels
        y_prob: the predicted probabilities
        ci: whether to calculate the confidence interval, by default is False
        n_bootstrap: the number of bootstrap samples, by default is 1000
        alpha: the significance level, by default is 0.05
    output:
        the metrics result as a dictionary, y_pred
    '''
    metrics_result = {}

    # Calculate metrics for the original dataset
    auc, recall, precision, accuracy, optimal_threshold, y_pred = calculate_sample(y_test, y_prob)

    # Store the original metrics
    metrics_result['Cutoff'] = optimal_threshold
    metrics_result['AUC'] = auc
    metrics_result['Recall'] = recall
    metrics_result['Precision'] = precision
    metrics_result['Accuracy'] = accuracy

    if ci:
        # Initialize lists to store bootstrapped metric results
        auc_bootstrap = []
        recall_bootstrap = []
        precision_bootstrap = []
        accuracy_bootstrap = []

        # Perform bootstrapping
        for seed in range(n_bootstrap):
            # Generate a bootstrapped sample
            np.random.seed(seed)
            indices = np.random.choice(len(y_test), size=len(y_test), replace=True)
            y_test = y_test.reset_index(drop=True)
            y_test_bootstrap = y_test[indices]
            y_prob_bootstrap = y_prob[indices]

            # Compute metrics for the bootstrapped sample
            auc_b, recall_b, precision_b, accuracy_b, _, _ = calculate_sample(y_test_bootstrap, y_prob_bootstrap)
            auc_bootstrap.append(auc_b)
            recall_bootstrap.append(recall_b)
            precision_bootstrap.append(precision_b)
            accuracy_bootstrap.append(accuracy_b)

        # Calculate confidence intervals and store in the format [lower, upper]
        metrics_result['AUC_CI'] = save_ci(auc_bootstrap, alpha)
        metrics_result['Recall_CI'] = save_ci(recall_bootstrap, alpha)
        metrics_result['Precision_CI'] = save_ci(precision_bootstrap, alpha)
        metrics_result['Accuracy_CI'] = save_ci(accuracy_bootstrap, alpha)

    return metrics_result, y_pred


## TensorFlow Model

In [34]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.losses import BinaryCrossentropy
import random

# Define the TensorFlow model
def create_model(input_dim, hidden_dim1, hidden_dim2, dropout_rate, l2_reg):
    model = Sequential([
        Dense(hidden_dim1, input_dim=input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_reg)),
        Dropout(dropout_rate),
        Dense(hidden_dim2, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_reg)),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    return model

def assess_mlp(X_train, X_test, y_train, y_test, selected_features, ci=False, plot_roc=False, calculate_shap=False, by_category=False):
    '''
    by default, the function will only return [0] metrics result
    ci: whether to calculate the confidence interval
    plot_roc: whether to plot the ROC curve
    calculate_shap: whether to calculate the SHAP values, if true, will return:
        [0] metrics result
        [1] shap values
        [2] base value
        [3] y_pred
    by_category: whether to calculate the AUC by each category, if true, will return:
        [0] metrics result
        [1] auc by each category
    if calculate_shap and by_category, will return:
        [0] metrics result
        [1] shap values
        [2] base value
        [3] y_pred
        [4] auc by each category
    '''

    seed = 42
    # Fixing the random seed
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    # Parameters
    hidden_dim1 = 16
    hidden_dim2 = 16
    dropout_rate = 0.008
    l2_reg = 0.001
    batch_size = 48
    epochs = 500
    learning_rate = 0.001


    auc_by_category = {0: [], 1: [], 2: []}

    # Split the data
    X_train, X_test = X_train.loc[:, selected_features], X_test.loc[:, selected_features]

    # Create and train the model
    model = create_model(input_dim=X_train.shape[1], hidden_dim1=hidden_dim1, hidden_dim2=hidden_dim2, dropout_rate=dropout_rate, l2_reg=l2_reg)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss=BinaryCrossentropy(), metrics=['AUC'])

    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

    # Predict and calculate AUC
    y_prob = model.predict(X_test, verbose=0).ravel()

    if by_category:
        for category in auc_by_category:
            test_mask = X_test['Surgery type'] == category
            train_mask = X_train['Surgery type'] == category

            if np.any(test_mask):
                X_test_filtered = X_test[test_mask]
                y_test_filtered = y_test[test_mask]

                # Predict and calculate AUC
                y_prob_filtered = model.predict(X_test_filtered).ravel()
                if len(np.unique(y_test_filtered)) > 1:
                    auc_category = roc_auc_score(y_test_filtered, y_prob_filtered)
                    auc_by_category[category].append(auc_category)


    if calculate_shap:
        # Compute SHAP values for the test set of this fold
        explainer = shap.DeepExplainer(model, np.array(X_train))
        base_value = explainer.expected_value[0]
        shap_value = explainer.shap_values(np.array(X_test))[0]

    # Calculate metrics
    fpr, tpr, thresholds = roc_curve(y_test, y_prob)
    metrics_result, y_pred = calculate_metrics(y_test, y_prob, ci=ci)
    metrics_result['Model'] = 'MLP'

    if plot_roc:
        plt.plot(fpr, tpr, label=f"MLP AUC: {metrics_result['AUC']:.4f}")

    returned = []
    returned.append(metrics_result)

    if calculate_shap:
        # compare the sum of shap values for each test sample and compare it with the output of the model
        returned.append(shap_value)
        returned.append(base_value)
        returned.append(y_pred)

    if by_category:
        returned.append(auc_by_category)

    return returned

## Traditional Models

In [35]:
traditional_models = [
    'LogisticRegression', 
    'LightGBM', 
    'XGBoost',
    'RandomForest',
    'SVM'
]

In [36]:
def assess_traditional(model_name, X_train, X_test, y_train, y_test, selected_features, ci = False, plot_roc = False, calculate_shap = False, by_category = False):

    '''
    by default, the function will only return [0] metrics result
    ci: whether to calculate the confidence interval
    plot_roc: whether to plot the ROC curve
    calculate_shap: whether to calculate the SHAP values, if true, will return:
        [0] metrics result
        [1] shap values
        [2] base value
        [3] y_pred
    by_category: whether to calculate the AUC by each category, if true, will return:
        [0] metrics result
        [1] auc by each category
    if calculate_shap and by_category, will return:
        [0] metrics result
        [1] shap values
        [2] base value
        [3] y_pred
        [4] auc by each category
    '''
    seed = 42
    random.seed(seed)
    np.random.seed(seed)

    auc_by_category = {0: [], 1: [], 2: []}

    X_train, X_test, = X_train.loc[:, selected_features], X_test.loc[:, selected_features]

    # sanitize column names to avoid error in LightGBM
    X_train.columns = X_train.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)
    X_test.columns = X_test.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)

    if model_name == 'LogisticRegression':
        model = LogisticRegression(random_state=seed)
    elif model_name == 'LightGBM':
        model = lgb.LGBMClassifier(random_state=seed, verbose=-1)
    elif model_name == 'XGBoost':
        model = xgb.XGBClassifier(random_state=seed, verbosity=0)
    elif model_name == 'RandomForest':
        model = RandomForestClassifier(random_state=seed)
    elif model_name == 'SVM':
        model = SVC(probability=True, random_state=seed)
    else:
        raise ValueError(f'Unknown model: {model_name}')
        

    model.fit(X_train, y_train)
    y_prob = model.predict_proba(X_test)[:, 1]
    
    if by_category:
        for category in auc_by_category:
            test_mask = X_test['Surgery_type'] == category

            if np.any(test_mask):
                X_test_filtered = X_test[test_mask]
                y_test_filtered = y_test[test_mask]

                # Predict and calculate AUC
                y_prob_filtered = model.predict_proba(X_test_filtered)[:, 1]

                if len(np.unique(y_test_filtered)) > 1:
                    auc_category = roc_auc_score(y_test_filtered, y_prob_filtered)
                    auc_by_category[category].append(auc_category)

    if calculate_shap:
        if model_name == 'LogisticRegression':
            explainer = shap.LinearExplainer(model, X_train)
        elif model_name in ['LightGBM', 'XGBoost', 'RandomForest']:
            explainer = shap.TreeExplainer(model)

        base_value = explainer.expected_value[0] if isinstance(explainer.expected_value, list) else explainer.expected_value
        shap_values = explainer.shap_values(X_test)
        shap_value = shap_values[1] if len(shap_values) == 2 else shap_values



    # use Youden's J Statistic to determine the optimal threshold
    fpr, tpr, thresholds = roc_curve(y_test, y_prob)
    metrics_result, y_pred = calculate_metrics(y_test, y_prob, ci=ci)
    metrics_result['Model'] = model_name

    if plot_roc:
        plt.plot(fpr, tpr, label=f"{model_name} AUC: {metrics_result['AUC']:.4f}")

    returned = []
    returned.append(metrics_result)
    
    if calculate_shap:
        returned.append(shap_value)
        returned.append(base_value)
        returned.append(y_pred)

    if by_category:
        returned.append(auc_by_category)


    return returned

# Model Explanation

## step 1: ROC curve

In [ ]:
# manually select the fold index
selected_index = 17966


plt.figure(figsize=(8, 8))

train_index, test_index = train_test_split(df.index, test_size=0.2, random_state=selected_index, stratify=df['aki'])
X_train, X_test = X.iloc[train_index], X.iloc[test_index]
y_train, y_test = y.iloc[train_index], y.iloc[test_index]

scaler = preprocessing.RobustScaler()
X_train.loc[:, numeric_noTime] = scaler.fit_transform(X_train[numeric_noTime])
X_test.loc[:, numeric_noTime] = scaler.transform(X_test[numeric_noTime])

qualified, spreadsheet_df = test_splitting(original_df, train_index, test_index)

metrics_result = assess_mlp(X_train, X_test, y_train, y_test, selected_features=X.columns, ci = True, plot_roc=True)[0]

metrics_ci = pd.DataFrame([metrics_result])

for model_name in traditional_models:
    model_result = assess_traditional(model_name, X_train, X_test, y_train, y_test, selected_features=X.columns, ci = True, plot_roc=True)[0]
    metrics_ci = pd.concat([metrics_ci, pd.DataFrame([model_result])], ignore_index=True)


# reorder the columns and save only 3 decimal places
metrics_ci = metrics_ci[['Model', 'AUC', 'AUC_CI', 'Recall', 'Recall_CI', 'Precision', 'Precision_CI', 'Accuracy', 'Accuracy_CI', 'Cutoff']]
metrics_ci = metrics_ci.round(3)

selected_model = 'LightGBM'


plt.plot([0, 1], [0, 1], linestyle='--', color='black')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Different Models')
plt.grid(False)
plt.legend()

# save the plot in a folder named 'plots_split_{selected_index}'
output_dir = f'plots_split_{selected_index}_{selected_model}'
os.makedirs(output_dir, exist_ok=True)
# clear all content in the folder
for item in os.listdir(output_dir):
    item_path = os.path.join(output_dir, item)
    if os.path.isfile(item_path):
        os.unlink(item_path)
    elif os.path.isdir(item_path):
        shutil.rmtree(item_path)
    
plt.savefig(f'plots_split_{selected_index}_{selected_model}/roc_curve.png', bbox_inches='tight')
metrics_ci.to_excel(f'plots_split_{selected_index}_{selected_model}/metrics_ci_{selected_index}.xlsx', index=False)
spreadsheet_df.to_excel(f'plots_split_{selected_index}_{selected_model}/train_test_split_{selected_index}.xlsx', index=False)


metrics_ci

## step 2: SHAP for all features

In [ ]:
# shap dot plot

result = assess_traditional(selected_model, X_train, X_test, y_train, y_test, selected_features=X.columns, calculate_shap=True)

metrics_result, shap_value, base_value, y_pred = result

# Generate SHAP summary plot
shap.summary_plot(shap_value, X_test, max_display=len(X.columns), plot_type='dot', show=False)

plt.savefig(f'plots_split_{selected_index}_{selected_model}/shap_summary_plot.png', bbox_inches='tight')


In [ ]:
# shap waterfall plot

# Create an Explanation object
patientNo = original_df['Patient No.']

shap_explanation = shap.Explanation(
    values=shap_value, 
    base_values= np.array([base_value[0] for _ in range(shap_value.shape[0])]),
    data= original_df[X.columns].iloc[test_index, :])

# plot all the waterfall plots and save the plots into a local folder named 'waterfall_plots'
output_dir = f'plots_split_{selected_index}_{selected_model}/waterfall_plots_all_features'
os.makedirs(output_dir, exist_ok=True)
# clear all content in the folder
for file in os.listdir(output_dir):
    os.remove(os.path.join(output_dir, file))

for i in range(len(shap_explanation)):
    # Generate the waterfall plot (SHAP will manage its own figure)
    shap.waterfall_plot(shap_explanation[i], max_display=12, show=False)

    # Get the current figure (created by SHAP)
    fig = plt.gcf()
    # Set the size of the figure
    fig.set_size_inches(12, 6)
    
    # Add a label to the plot

    label = f'Waterfall Plot of Patient No. {patientNo.iloc[test_index[i]]}: aki {int(y.iloc[test_index[i]])} prediction {int(y_pred[i])}'
    plt.text(0.5, 0.95, label, 
             ha='center', va='center', transform=fig.transFigure, fontsize=16, fontweight='bold')
    # Save the current figure
    img_name = f'{output_dir}/aki_{int(y.iloc[test_index[i]])}_prediction_{int(y_pred[i])}_patientNo_{patientNo.iloc[test_index[i]]}.png'
    fig.savefig(img_name, bbox_inches='tight')
    # Close the figure to free up memory
    plt.close(fig)

print('SHAP waterfall plots for all features are saved in the folder: ', output_dir)

## step 3: feature selection curve

In [ ]:
# Get feature importance
mean_shap_values = np.abs(shap_value).mean(axis=0)
sorted_indices = np.argsort(mean_shap_values)
sorted_indices = np.flip(sorted_indices)
sorted_features = X.columns[sorted_indices]

feature_selection_auc = []

for num_features in range(1, len(X.columns)+1):
    metrics_result = assess_traditional(selected_model, X_train, X_test, y_train, y_test, selected_features=sorted_features[:num_features])

    auc = metrics_result[0]['AUC']

    feature_selection_auc.append(auc)  

# find the highest AUC and its column name in each row
highest_auc = np.max(feature_selection_auc)
highest_num = feature_selection_auc.index(highest_auc) + 1

# find the list of features for the highest AUC and save it in the column named 'Best_Remaining_Features'
best_features = sorted_features[:highest_num].tolist()
best_features

In [ ]:
# plot the feature selection curve

# Set the style for the chart
sns.set(style="whitegrid", palette="deep", font_scale=1.2)

# Create the figure and axis
plt.figure(figsize=(8, 6))

# Plotting with straight lines and dots
plt.plot(range(1, len(X.columns)+1), feature_selection_auc, marker='o', linestyle='-', color='b', markersize=6, linewidth=2)

# Set axis labels and chart title
plt.xlabel("Number of Features", fontsize=14)
plt.ylabel("AUC", fontsize=14)
plt.title("AUC vs Number of Features", fontsize=16)

plt.legend()
plt.tight_layout()

# save the plot in a folder named 'plots_split_{selected_index}'

plt.savefig(f'plots_split_{selected_index}_{selected_model}/feature_selection_curve.png', bbox_inches='tight')


## step 4: SHAP for best remaining features

In [ ]:
# find the best remaining features
best_with_intraoperative = best_features + intraoperative_features

final_numeric = [col for col in numeric_cols if col in best_with_intraoperative]

X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(new_X.loc[:, best_with_intraoperative], y, test_size=0.2, random_state=selected_index, stratify=y)

scaler = preprocessing.RobustScaler()
X_train_final.loc[:, final_numeric] = scaler.fit_transform(X_train_final.loc[:, final_numeric])
X_test_final.loc[:, final_numeric] = scaler.transform(X_test_final.loc[:, final_numeric])

result = assess_traditional(selected_model, X_train_final, X_test_final, y_train_final, y_test_final, selected_features=best_with_intraoperative, calculate_shap=True, by_category=True)

metrics_result_best, shap_value_best, base_value_best, y_pred_best, auc_by_category_best = result

# Generate SHAP summary plot
shap.summary_plot(shap_value_best, X_test_final, max_display=len(best_with_intraoperative), plot_type='dot')

# save the plot
plt.savefig(f'plots_split_{selected_index}_{selected_model}/shap_summary_plot_best.png', bbox_inches='tight')

print('AUC by category: ', auc_by_category_best)


In [ ]:
# shap waterfall plot

# Create an Explanation object
patientNo = original_df['Patient No.']

shap_explanation_best = shap.Explanation(
    values=shap_value_best, 
    base_values= np.array([base_value_best[0] for _ in range(shap_value_best.shape[0])]), 
    data= original_df[best_with_intraoperative].iloc[test_index, :])


# plot all the waterfall plots and save the plots into a local folder named 'waterfall_plots'
output_dir = f'plots_split_{selected_index}_{selected_model}/waterfall_plots_best_features+intraoperative'
os.makedirs(output_dir, exist_ok=True)
# clear all content in the folder
for file in os.listdir(output_dir):
    os.remove(os.path.join(output_dir, file))


for i in range(len(shap_explanation_best)):
    # Generate the waterfall plot (SHAP will manage its own figure)
    shap.waterfall_plot(shap_explanation_best[i], max_display=12, show=False)

    # Get the current figure (created by SHAP)
    fig = plt.gcf()
    # Set the size of the figure
    fig.set_size_inches(12, 6)
    
    # Add a label to the plot

    label = f'Waterfall Plot of Patient No. {patientNo.iloc[test_index[i]]}: aki {int(y.iloc[test_index[i]])} prediction {int(y_pred_best[i])}'
    plt.text(0.5, 0.95, label, 
             ha='center', va='center', transform=fig.transFigure, fontsize=16, fontweight='bold')
    # Save the current figure
    img_name = f'{output_dir}/aki_{int(y.iloc[test_index[i]])}_prediction_{int(y_pred_best[i])}_patientNo_{patientNo.iloc[test_index[i]]}.png'
    fig.savefig(img_name, bbox_inches='tight')
    # Close the figure to free up memory
    plt.close(fig)

print('SHAP waterfall plots for final features are saved in the folder: ', output_dir)


In [45]:
# Generate SHAP dependence plots for each numerical feature
n_cols = 3
n_rows = (len(final_numeric) + n_cols - 1) // n_cols  # Calculate the number of rows needed
fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 5, n_rows * 5))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Define labels for each subplot
labels = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")

for i, feature in enumerate(final_numeric):
    sns.scatterplot(x=original_df.iloc[test_index, :][feature], y=shap_value_best[:, best_with_intraoperative.index(feature)], ax=axes[i], hue=y_test_final)
    axes[i].set_title(f'SHAP Dependence Plot for {feature}')
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('SHAP Value')
    axes[i].grid(False)
    
    # Add label to the subplot
    axes[i].text(-0.05, 1.1, labels[i], transform=axes[i].transAxes, 
                 fontsize=20, fontweight='bold', va='top', ha='right')
# Remove any empty subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()

# Save the plot
plt.savefig(f'plots_split_{selected_index}_{selected_model}/shap_dependence_plot_best.png', bbox_inches='tight')
